# Predicting NBA Game Outcomes with Machine Learning
## Author: Kishore Annambhotla

### Introduction and Methods


### Setting Up

In order to predict NBA game outcomes, we will need to analyze many past games and see what statistics help or hurt a team's chances of winning. We will also need a robust predictive model capable of making a decision based on all of this game data. For this project, I used `nba_api` to retrieve up-to-date NBA game data, `pandas` and `numpy` for data manipulation, and `scikit-learn` to build an ML model. All of the project's necessary imports will be included in the code block below.

In [1]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd

pd.options.mode.chained_assignment = (
    None  # prevents warnings on reformatted columns later
)
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

### Importing Data

We will base our model off of games played by all NBA teams. As shown, this game data will include categorical information about the game, teams, and season, as well as quantitative data regarding each team's offense and defense, as well as the winner of the game.

First, we will use the `nba_api` to get games played by each NBA team. Note that we set `season_type_nullable` to only include regular season games, since those are the games we will be predicting with this model. This could easily be changed to train a model better for predicting playoff or preseason games.

In [2]:
nba_teams = teams.get_teams()
team_abbr_to_id = {team["abbreviation"]: team["id"] for team in nba_teams}
all_games = pd.DataFrame()

for team in nba_teams:
    team_id = team["id"]
    gamefinder = leaguegamefinder.LeagueGameFinder(
        team_id_nullable=team_id, season_type_nullable="Regular Season"
    )
    games = gamefinder.get_data_frames()[0]
    all_games = pd.concat([all_games, games], ignore_index=True)

In [3]:
print(all_games.columns)
print(all_games.sample(n=5))

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'],
      dtype='object')
      SEASON_ID     TEAM_ID TEAM_ABBREVIATION            TEAM_NAME  \
5433      22000  1610612738               BOS       Boston Celtics   
7970      22010  1610612739               CLE  Cleveland Cavaliers   
69445     22016  1610612758               SAC     Sacramento Kings   
94688     22018  1610612766               CHA    Charlotte Hornets   
56372     22009  1610612754               IND       Indiana Pacers   

          GAME_ID   GAME_DATE      MATCHUP WL  MIN  PTS  ...  FT_PCT  OREB  \
5433   0020000351  2000-12-18  BOS vs. DAL  L  240   98  ...   0.840  16.0   
7970   0021000881  2011-02-27  CLE vs. PHI  L  240   91  ...   0.667  11.0   
69445  0021601140  2017-04-01    SA

While this data is good, it may be too much for our model. Over the last forty seasons, NBA basketball has greatly changed. Many rules have been added and removed, and aspects of "perfect" offense and defense have completely shifted. As a result, older data is increasingly unreliable for predicting current-day NBA games. For this model, I chose to only use data from the 2018-2019 season until now. This time period is most reflective of modern basketball, and ignores plenty of outdated game data.

In [4]:
# Get games where season is in modern_seasons
games_modern = all_games[
    (all_games.SEASON_ID.str[-4:] == "2018")
    | (all_games.SEASON_ID.str[-4:] == "2019")
    | (all_games.SEASON_ID.str[-4:] == "2020")
    | (all_games.SEASON_ID.str[-4:] == "2021")
    | (all_games.SEASON_ID.str[-4:] == "2022")
    | (all_games.SEASON_ID.str[-4:] == "2023")
    | (all_games.SEASON_ID.str[-4:] == "2024")
]
print(games_modern.sample(n=5))

      SEASON_ID     TEAM_ID TEAM_ABBREVIATION            TEAM_NAME  \
81569     22023  1610612762               UTA            Utah Jazz   
91154     22020  1610612765               DET      Detroit Pistons   
69130     22020  1610612758               SAC     Sacramento Kings   
331       22020  1610612737               ATL        Atlanta Hawks   
6988      22022  1610612739               CLE  Cleveland Cavaliers   

          GAME_ID   GAME_DATE      MATCHUP WL  MIN  PTS  ...  FT_PCT  OREB  \
81569  0022300835  2024-02-27    UTA @ ATL  L  240   97  ...   0.842  12.0   
91154  0022000165  2021-01-13  DET vs. MIL  L  240  101  ...   0.682  13.0   
69130  0022000789  2021-04-08  SAC vs. DET  L  241  101  ...   0.692   6.0   
331    0022000705  2021-03-28    ATL @ DEN  L  241  102  ...   0.857   2.0   
6988   0022200631  2023-01-12    CLE @ POR  W  240  119  ...   0.774  12.0   

       DREB   REB  AST   STL  BLK  TOV  PF  PLUS_MINUS  
81569  31.0  43.0   24  11.0    4   17  15       -27.

### Cleaning Data

We have plenty of good data, but it's still unrefined. The next step is to clean up our columns and reformat the data. This will make training, testing, and evaluating our model much easier in the future.

First, we will reformat some of our columns to make working with the data easier.

In [5]:
# Convert GAME_DATE to pandas datetime
# Also order by date earliest to latest to make working with stats easier later
games_modern["GAME_DATE"] = pd.to_datetime(games_modern["GAME_DATE"])
games_modern.sort_values(by="GAME_DATE", inplace=True)


# Add binary "WIN" column, remove categorical WL column
games_modern["WIN"] = games_modern["WL"].apply(lambda x: 1 if x == "W" else 0)


# Convert int stat columns to float type for accurate data analysis
games_modern["MIN"] = games_modern["MIN"].astype(float)  # minutes
games_modern["PTS"] = games_modern["PTS"].astype(float)  # points
games_modern["FGM"] = games_modern["FGM"].astype(float)  # field goals made
games_modern["FGA"] = games_modern["FGA"].astype(float)  # field goals attempted
games_modern["FG3M"] = games_modern["FG3M"].astype(float)  # 3s made
games_modern["FG3A"] = games_modern["FG3A"].astype(float)  # 3s attempted
games_modern["FTM"] = games_modern["FTM"].astype(float)  # free throws made
games_modern["FTA"] = games_modern["FTA"].astype(float)  # free throws attempted
games_modern["AST"] = games_modern["AST"].astype(float)  # assists
games_modern["BLK"] = games_modern["BLK"].astype(float)  # blocks
games_modern["TOV"] = games_modern["TOV"].astype(float)  # turnovers
games_modern["PF"] = games_modern["PF"].astype(float)  # personal fouls


# Add opponent id as column


def get_opponent_id(matchup, team_abbr_to_id, team_id):
    if "@" in matchup:

        opponent_abbr = matchup.split(" @ ")[-1]

    else:

        opponent_abbr = matchup.split(" vs. ")[-1]

    return team_abbr_to_id.get(opponent_abbr, team_id)


games_modern["OPP_TEAM_ID"] = games_modern.apply(
    lambda row: get_opponent_id(row["MATCHUP"], team_abbr_to_id, row["TEAM_ID"]), axis=1
)

Next, we want to define a few new statistics that will be useful in evaluating a team. These statistics will prove essential in predicting the outcomes of new games. For many of these statistics, we will only want to create them based on the most recent data to avoid inaccuracies. Because teams can undergo vast changes in the span of two or three years, we will only define these statistics using data over the last two seasons.

In [6]:
# Define 'HGA' (Home Game Advantage)
games_modern["HGA"] = games_modern["MATCHUP"].apply(lambda x: 0 if "@" in x else 1)

# Define 'LAST_GAME_OUTCOME'
games_modern["LAST_GAME_OUTCOME"] = (
    games_modern.groupby("TEAM_ID")["WIN"].shift(1).fillna(0)
)

# Note: per game statistics indicate team per game statistics using rolling average of last 10
# Define 'PPG' (Points Per Game)
# games_modern["PPG"] = (
#    games_modern.groupby("TEAM_ID")["PTS"]
#    .rolling(window=10, closed="left", min_periods=1)
#    .mean()
#    .reset_index(0, drop=True)
# )

# Define 'EFG%' (Effective Field Goal Percentage)
games_modern["EFG%"] = (
    games_modern["FGM"] + (0.5 * games_modern["FG3M"])
) / games_modern["FGA"]

# Define 'TOV%' (Turnover Percentage)
games_modern["TOV%"] = games_modern["TOV"] / (
    games_modern["FGA"] + 0.44 * games_modern["FTA"] + games_modern["TOV"]
)


# Define 'ORB%' (Offensive Rebound Percentage)
def get_opponent_dreb(game_id):
    opponent_dreb = games_modern.loc[games_modern["GAME_ID"] == game_id, "DREB"].values[0]
    return opponent_dreb


games_modern["ORB%"] = games_modern["OREB"] / (
    games_modern["OREB"] + get_opponent_dreb(games_modern["GAME_ID"])
)

# Define 'FTR' (Free Throw Attempt Rate)
games_modern["FTR"] = games_modern["FTA"] / games_modern["FGA"]

# Define 'TS%' (True Shooting Percentage)
games_modern["TS%"] = games_modern["PTS"] / (
    2 * (games_modern["FGA"] + (0.44 * games_modern["FTA"]))
)

Finally, we want to use a `LabelEncoder` to transform a few of our categorical data columsn into numerical values that can be understood by our model.

In [7]:
# LabelEncode data
le = LabelEncoder()
games_modern["TEAM_ID"] = le.fit_transform(games_modern["TEAM_ID"])
games_modern["OPP_TEAM_ID"] = le.fit_transform(games_modern["OPP_TEAM_ID"])
games_modern.sample(n=5)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,PLUS_MINUS,WIN,OPP_TEAM_ID,HGA,LAST_GAME_OUTCOME,EFG%,TOV%,ORB%,FTR,TS%
36230,22022,11,MIA,Miami Heat,0022200628,2023-01-12,MIA vs. MIL,W,240.0,108.0,...,6.0,1,12,1,1.0,0.489474,0.113004,0.428571,0.168421,0.529204
32969,22020,10,LAL,Los Angeles Lakers,0022000630,2021-03-18,LAL vs. CHA,W,240.0,116.0,...,11.0,1,29,1,1.0,0.548913,0.098390,0.314286,0.217391,0.575397
75717,22023,23,OKC,Oklahoma City Thunder,0022301069,2024-03-29,OKC vs. PHX,W,240.0,128.0,...,25.0,1,19,1,0.0,0.655172,0.096937,0.225806,0.160920,0.686990
79168,22023,24,TOR,Toronto Raptors,0022300492,2024-01-05,TOR @ SAC,L,240.0,130.0,...,-5.0,0,21,0,1.0,0.614943,0.124026,0.250000,0.310345,0.657362
19060,22023,6,DEN,Denver Nuggets,0022300226,2023-11-22,DEN @ ORL,L,240.0,119.0,...,-5.0,0,16,0,1.0,0.553763,0.089095,0.414634,0.225806,0.581964


### Defining Features

In our model, `X` includes features of the dataset that will be used to predict `y`. Here, `y` is obviously `WIN`, since we are just predicting and classifying each game as a win (1) or loss (0). `X` could include a number of relevant statistics, and all the ones used will be included in the code segment.

In [8]:
features = [
    "TEAM_ID",
    "OPP_TEAM_ID",
    "PTS",
    "OREB",
    "DREB",
    "REB",
    "AST",
    "STL",
    "BLK",
    "TOV",
    "EFG%",
    "TOV%",
    "ORB%",
    "FTR",
    "TS%",
    "HGA",
    "LAST_GAME_OUTCOME",
]
X = games_modern[features]

y = games_modern[["WIN"]]


# Use sklearn train_test_split with 80% training, 20% testing

X_train, X_test, y_train, y_test = train_test_split(

    X, y, test_size=0.2, random_state=42

)

### Creating Initial Model

`scikit-learn` offers many options for ML classification models. A simple one good for this problem would be a `RandomForestClassifier`, which uses many decision trees to analyze data and settle on a choice. Random forests are known for accuracy and efficiency.

In [9]:
rf_model = RandomForestClassifier(n_estimators=150, max_depth=20, random_state=42)
rf_model.fit(X_train, y_train)

c:\Users\14846\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(max_depth=20, n_estimators=150, random_state=42)

In [11]:
y_pred = rf_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.819435928877989
              precision    recall  f1-score   support

           0       0.80      0.85      0.82      1628
           1       0.84      0.79      0.81      1634

    accuracy                           0.82      3262
   macro avg       0.82      0.82      0.82      3262
weighted avg       0.82      0.82      0.82      3262



### Predicting Games